In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import requests

In [ ]:
spark = SparkSession.builder \
    .appName("Aula Spark 13DTRS") \
    .getOrCreate()

In [ ]:
df_futebol = spark.read.csv("results.csv", header=True, inferSchema=True)

df_futebol.printSchema()
df_futebol.show(5)

In [ ]:
df_futebol_ok =(
  df_futebol
  .withColumnRenamed("date", "dt")
  .withColumnRenamed("home_teamName", "mandante")
  .withColumnRenamed("away_teamName", "visitante")
  .withColumnRenamed("home_scoreHome", "gol_mandante")
  .withColumnRenamed("away_scoreAway", "gol_visitante")
  .withColumnRenamed("tournamentName", "torneio")
  .withColumnRenamed("cityCity", "cidade")
  .withColumnRenamed("countryCountry", "pais")
  .withColumnRenamed("neutral", "neutro")
  .drop('neutralTRUE')
  .withColumn("dt",col("dt").cast('date'))
  .withColumn("gol_mandante",col("gol_mandante").cast('integer'))
  .withColumn("gol_visitante",col("gol_visitante").cast('integer'))

)

In [ ]:
df_futebol_ok.show()

In [ ]:
df_futebol_ok.createOrReplaceTempView("futebol")

In [ ]:
#1 - Quantos registros existem na base?
query_count = "SELECT COUNT(*) from futebol"
count_df = spark.sql(query_count)
count_df.show()

In [ ]:
# 2 - Quantas equipes únicas mandantes existem na base?
query = "SELECT COUNT(DISTINCT mandante) FROM futebol"
unique_home_teams_df = spark.sql(query)
unique_home_teams_df.show()

In [ ]:
# 3 - Quantas vezes as equipes mandantes saíram vitoriosas?
query_home_wins = "SELECT COUNT(*) FROM futebol WHERE gol_mandante > gol_visitante"
home_wins_df = spark.sql(query_home_wins)
home_wins_df.show()

In [ ]:
# 4 - Quantas vezes as equipes visitantes saíram vitoriosas?
query_away_wins = "SELECT COUNT(*) FROM futebol WHERE gol_visitante > gol_mandante"
away_wins_df = spark.sql(query_away_wins)
away_wins_df.show()

In [ ]:
#5 - Quantas partidas resultaram em empate?
query_draws = "SELECT COUNT(*) FROM futebol WHERE gol_mandante = gol_visitante"
draws_df = spark.sql(query_draws)
draws_df.show()

In [ ]:
#6 — Quantas partidas foram real izadas em cada país?
query_play = ('SELECT pais, COUNT(*) AS total_partidas FROM futebol GROUP BY pais ORDER BY total_partidas DESC')
play_df = spark.sql(query_play)
play_df.show()

In [ ]:
#7 - Qual país teve mais partidas?
query_max_play = ('SELECT pais, COUNT(*) AS total_partidas FROM futebol GROUP BY pais ORDER BY total_partidas DESC LIMIT 1')
max_play_df = spark.sql(query_max_play)
max_play_df.show()

In [ ]:
#8 - Qual a partida com maior número de gols?
query_max_goals = ('SELECT dt, mandante, visitante, gol_mandante, gol_visitante, cidade, pais,(gol_mandante + gol_visitante) AS total_gols FROM futebol ORDER BY total_gols DESC LIMIT 1')
max_goals_df = spark.sql(query_max_goals)
max_goals_df.show()

In [ ]:
#9 - Qual a maior goleada?
query_max_goals = ('SELECT dt, mandante, visitante, gol_mandante, gol_visitante, cidade, pais, ABS(gol_mandante- gol_visitante) AS diferenca_gols FROM futebol ORDER BY diferenca_gols DESC LIMIT 1')
max_goals_df = spark.sql(query_max_goals)
max_goals_df.show()

In [ ]:
#10 - Quantos jogos ocorreram no Brasil?
query_brasil = ('SELECT COUNT(*) AS jogos_no_brasil FROM futebol WHERE pais = "Brazil";')
brasil_df = spark.sql(query_brasil)
brasil_df.show()